In [27]:
import numpy as np
import math
import copy
import sys
np.set_printoptions(suppress=True, linewidth=np.nan)


# Adjacent Graph with simplex method

### Utilities

In [28]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[[c], A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [29]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = math.inf
    for k in range(len(b)):
        if col[k] > 0:
            if min_val > (min_val_temp := b[k] / col[k]):
                min_val = min_val_temp
                r_min = k
    return r_min


In [30]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    tableau[row] = tableau[row] / tableau[row, col]
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]


In [31]:
def get_vertex(tableau: np.ndarray, basic_var: list[int]) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1  # for Big-M don't delete artificial variables
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    print("Vertex: x =", x[:n])


### Simplex Algorithm

In [32]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = math.inf
    for k in range(len(b)):
        if col[k] > 0:
            if min_val > (min_val_temp := b[k] / col[k]):
                min_val = min_val_temp
                r_min = k
    return r_min


In [36]:
def minimum_ratio_test_multiple(col: np.ndarray, b: np.ndarray) -> int:
    map_test: dict[float, list[int]] = {}
    min_key = np.Inf
    for k in range(len(b)):
        if col[k] > 0:
            test = b[k] / col[k]
            if test in map_test:
                map_test[test].append(k + 1)
            else:
               map_test[test] = [k + 1]
            min_key = min(min_key, test)
    return map_test[min_key]

In [47]:
def adjacent_graph(tableau: np.ndarray, basic_var: list[int], row: int = None, col: int = None) -> None:
    get_vertex(tableau, basic_var)
    print(f"Pivoting = {row, col}, x_B = {basic_var}\n{tableau}")
    if (tableau[0, :-1] > 0).any():
        # recursive simplex for each column candidate
        for col in np.where(tableau[0, :-1] > 0)[0]:
            # minimum ratio test
            multi_row = minimum_ratio_test_multiple(col=tableau[1:, col], b=tableau[1:, -1]) 
            for row in multi_row:
                # temp array
                temp = copy.deepcopy(tableau)
                temp_basic_var = copy.deepcopy(basic_var)
                # pivoting
                pivoting(temp, row, col)
                # swap row with col
                temp_basic_var[row - 1] = col
                # recursive proccess
                adjacent_graph(temp, temp_basic_var, row, col)
    else:
        print("End proccess")


In [48]:
A = [[1, 1, 1, 0, 0, 0],
     [3, 1, 0, 1, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1.]]

b = [9, 18, 7, 6.]

c = [3, 2, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5]

tableau = build_tableau(A, b, c)
adjacent_graph(tableau, basic_var)

Vertex: x = [0. 0.]
Pivoting = (None, None), x_B = [2, 3, 4, 5]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
Vertex: x = [6. 0.]
Pivoting = (2, 0), x_B = [2, 0, 4, 5]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
Vertex: x = [4.5 4.5]
Pivoting = (1, 1), x_B = [1, 0, 4, 5]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0

In [59]:
def correct_tableau(tableau: np.ndarray, basic_var: list[int]):
    # correct first row of tableau because the artificial_var and min=-max position is negative
    for index, col in enumerate(basic_var):
        row = index + 1
        tableau[row] = tableau[row] / tableau[row, col]
        for k in range(len(tableau)):
            if k != row:
                tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]
    return tableau


In [60]:
A = [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1.]]

b = [1, 1.]

c = [0, 0, 1, 0, 0.]

# basic_var = [3, 4]
basic_var_list = [[3, 4], [1, 4], [0, 4], [0, 1], [0, 3]]

for basic_var in basic_var_list:
     print("==============New Simplex Proccess==============")
     tableau = build_tableau(A, b, c)
     tableau = correct_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var)

==============New Simplex Proccess==============
Vertex: x = [0. 0. 0.]
Pivoting = (None, None), x_B = [3, 4]
[[0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1.]]
Vertex: x = [0. 0. 1.]
Pivoting = (1, 2), x_B = [2, 4]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
End proccess
==============New Simplex Proccess==============
Vertex: x = [0. 1. 0.]
Pivoting = (None, None), x_B = [1, 4]
[[0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1.]]
Vertex: x = [0. 0. 1.]
Pivoting = (1, 2), x_B = [2, 4]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
End proccess
==============New Simplex Proccess==============
Vertex: x = [1. 0. 0.]
Pivoting = (None, None), x_B = [0, 4]
[[ 0.  0.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 0. -1. -1. -1.  1.  0.]]
Vertex: x = [0. 0. 1.]
Pivoting = (1, 2), x_B = [2, 4]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
End proccess
==========

In [61]:
A = [[-2, 6, 1, 0, 0],
     [2, 4., 0, 1, 0],
     [1, 0., 0, 0, 1]]

b = [0, 10, 4.]

c = [-1, 1, 0, 0, 0.]

# basic_var = [3, 4]
basic_var_list = [[0, 3, 2], [0, 1, 2]]

for basic_var in basic_var_list:
     print("==============New Simplex Proccess==============")
     tableau = build_tableau(A, b, c)
     tableau = correct_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var)

==============New Simplex Proccess==============
Vertex: x = [4. 0.]
Pivoting = (None, None), x_B = [0, 3, 2]
[[ 0.  1.  0.  0.  1.  4.]
 [ 1.  0.  0.  0.  1.  4.]
 [ 0.  4.  0.  1. -2.  2.]
 [ 0.  6.  1.  0.  2.  8.]]
Vertex: x = [4.  0.5]
Pivoting = (2, 1), x_B = [0, 1, 2]
[[ 0.    0.    0.   -0.25  1.5   3.5 ]
 [ 1.    0.    0.    0.    1.    4.  ]
 [ 0.    1.    0.    0.25 -0.5   0.5 ]
 [ 0.    0.    1.   -1.5   5.    5.  ]]
Vertex: x = [3. 1.]
Pivoting = (3, 4), x_B = [0, 1, 4]
[[ 0.   0.  -0.3  0.2  0.   2. ]
 [ 1.   0.  -0.2  0.3  0.   3. ]
 [ 0.   1.   0.1  0.1  0.   1. ]
 [ 0.   0.   0.2 -0.3  1.   1. ]]
Vertex: x = [0. 0.]
Pivoting = (1, 3), x_B = [3, 1, 4]
[[-0.66666667  0.         -0.16666667  0.          0.          0.        ]
 [ 3.33333333  0.         -0.66666667  1.          0.         10.        ]
 [-0.33333333  1.          0.16666667  0.          0.          0.        ]
 [ 1.          0.          0.          0.          1.          4.        ]]
End proccess
Vertex: x 